# Players dataset preparation

In [1]:
import pandas as pd
import numpy as np

# Visualization
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = 'plotly'
pio.templates.default = "seaborn"

## Dataset preparation

In [2]:
# Read datasets
df_tennis = pd.read_csv("./datasets/tennis_matches_cleaned.csv", parse_dates=["tourney_date"], index_col=0)
df_male = pd.read_csv("./datasets/male_players.csv")
df_female = pd.read_csv("./datasets/female_players.csv")

df_male.drop_duplicates(inplace=True)
df_female.drop_duplicates(inplace=True)

def preprocess_strings(df):
    df = df.applymap(lambda x:x.lower().strip() if type(x) == str else x)
    return df.replace(r"\s{2,}", " ", regex=True)

df_tennis.tourney_date = pd.to_datetime(df_tennis.tourney_date, format='%Y%m%d')

# Normalize strings
df_male = preprocess_strings(df_male)
df_female = preprocess_strings(df_female)

In [3]:
# Create a new players dataset whose name is the concatenation of the name and the surname for the male and female players
df = pd.concat([df_male.name + " " + df_male.surname], axis=1, keys=["name", "gender"])
df["gender"] = "m"
df_players = df
df = pd.concat([df_female.name + " " + df_female.surname], axis=1, keys=["name", "gender"])
df["gender"] = "f"
df_players = df_players.append(df)

# Remove duplicates arising from the intersection
df_players = df_players.drop_duplicates(subset=["name"])
df_players = df_players.sort_values(by=["name"])
df_players.reset_index(drop=True, inplace=True)

# Merge the players dataset with the tennis dataset
df = pd.merge(df_tennis, df_players.rename(columns = {'name': 'winner_name', 'gender':'winner_gender'}), on="winner_name", how="left")
df = pd.merge(df, df_players.rename(columns = {'name': 'loser_name', 'gender':'loser_gender'}), on="loser_name", how="left")

# Find the players who do not have a gender and assign them the most common among the genders of the players they played with. 
winners = df[df.winner_gender.isnull()].groupby(["winner_name", "loser_gender"]).loser_gender.count().reset_index(name="occurrences").rename(columns={"winner_name":"name", "loser_gender":"gender"})

losers = df[df.loser_gender.isnull()].groupby(["loser_name", "winner_gender"]).winner_gender.count().reset_index(name="occurrences").rename(columns={"loser_name":"name", "winner_gender":"gender"})

estimate = pd.concat([winners, losers]).groupby(["name", "gender"]).occurrences.sum().reset_index(name="occurrences")
estimate = estimate.sort_values(by=["name", "occurrences"], ascending=[1, 0]).drop_duplicates("name", keep="first").drop(columns=["occurrences"])

# Create the players dataset
df_players = df.loc[:, ['winner_name', 'winner_gender']].set_axis(['name', 'gender'], axis=1, inplace=False)
df_players = df_players.append(df.loc[:, ['loser_name', 'loser_gender']].set_axis(['name', 'gender'], axis=1, inplace=False))
df_players = df_players.append(estimate)
df_players = df_players.dropna().drop_duplicates()
df_players = df_players.sort_values(by=["name"]).reset_index(drop=True)
df_players.head()

,name,gender
0,aada inna,f
1,aalisha alexis,f
2,aaliya ebrahim,f
3,aaliyah hohmann,f
4,aalyka ebrahim,f


## Feature engineering

### Tourneys played

In [4]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id']].rename(columns={'loser_name': 'name'})

# players and their number of participations in tournaments
df = pd.concat([df_w, df_l]).groupby('name').tourney_id.nunique().reset_index(name="total_tourneys_played")

# add features to the data_frame of player profiles
df_players = df_players.merge(df, on="name")

### Matches played

In [5]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id']].rename(columns={'loser_name': 'name'})

# players and relative number of matches played
df = pd.concat([df_w, df_l]).groupby('name').tourney_id.count().reset_index(name="total_matches_played")

# add features to the player profile dataframe
df_players = df_players.merge(df, on="name")

### Matches won

In [6]:
df = df_tennis.loc[:, ['winner_name', 'tourney_id']].rename(columns={'winner_name': 'name'})

# players and relative number of matches won
df = df.groupby('name').tourney_id.count().reset_index(name="total_matches_won")

# add features to the data_frame of player profiles
df_players = df_players.merge(df, on="name", how="left")
df_players.total_matches_won.fillna(0, inplace=True)

### Matches won ratio

In [7]:
df_players["matches_won_ratio"] = df_players["total_matches_won"] / df_players["total_matches_played"]

### Mean, max, min, entropy of performance_index

It's the number of matches won in a tourney over the matches that were supposed to be played in order to win the tourney

In [8]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_id', "draw_size"]].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'tourney_id', "draw_size"]].rename(columns={'loser_name': 'name'})

matches_won_per_tourney = df_w.groupby(['name', 'tourney_id']).tourney_id.count().reset_index(name="total_matches_won_per_tourney")
df = pd.concat([df_w, df_l])
df = df.merge(matches_won_per_tourney, on=['name', 'tourney_id'], how="left")
df["total_matches_won_per_tourney"] = df["total_matches_won_per_tourney"].fillna(0).astype('Int32')

df["matches_to_play_per_tourney"] = np.log2(df["draw_size"]) + 1
df["performance_index"] = df["total_matches_won_per_tourney"] / df["matches_to_play_per_tourney"]
# If performance_index is bigger than 1 (won more matches than the number of matches I am supposed to play), then set it to 1. That's the case of extra qualifications
df["performance_index"].clip(lower=0, upper=1, inplace=True)

# Stats about performance_index
df_stats = df.groupby('name')['performance_index'].agg(mean_performance_index='mean', max_performance_index='max', min_performance_index='min')

# Entropy of performance_index
g_sum = df.groupby('name')['performance_index'].transform('sum')
values = df['performance_index']/g_sum
df['performance_index_entropy'] = -(values*np.log(values))
df_entropy = df.groupby('name')['performance_index_entropy'].sum().reset_index()

# Merge stats and entropy with players
df_players = df_players.merge(df_stats, on="name")
df_players = df_players.merge(df_entropy, on="name")
#del df_entropy, df_stats

C:\Users\Domenico\Anaconda3\envs\dm\lib\site-packages\pandas\core\arrays\numeric.py:203: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs2, **kwargs)


### Height

Probably unusable feature, we only have it for 541 players

In [9]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_ht', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_ht': 'ht'})
df_l = df_tennis.loc[:, ['loser_name', 'loser_ht', 'tourney_date']].rename(columns={'loser_name': 'name', 'loser_ht': 'ht'})

# players and their height
df = pd.concat([df_w, df_l]).sort_values('tourney_date', ascending='False').groupby('name').ht.first().reset_index()

# add features to the data_frame of player profiles
df_players = df_players.merge(df, on="name")

### Age

In [10]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_age', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_age': 'age'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_age', 'tourney_date']]).rename(columns={'loser_name': 'name', 'loser_age': 'age'})

# players and their ages
df = pd.concat([df_w, df_l]).sort_values('tourney_date', ascending='False').groupby('name').age.first().reset_index()

# 
df_players = df_players.merge(df, on="name")


In [33]:
px.histogram(df_players, x="age").show()

### Hand

In [11]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_hand']].rename(columns={'winner_name': 'name', 'winner_hand': 'hand'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_hand']]).rename(columns={'loser_name': 'name', 'loser_hand': 'hand'})

# players and their favorite hand
df = pd.concat([df_w, df_l]).groupby('name').hand.agg(pd.Series.mode).reset_index()

# add features to the player profile dataframe
df_players = df_players.merge(df, on="name")

### Mean, max, entropy of minutes

In [12]:
df_w = df_tennis.loc[:, ['winner_name', 'minutes']].rename(columns={'winner_name': 'name'})
df_l = df_tennis.loc[:, ['loser_name', 'minutes']].rename(columns={'loser_name': 'name'})

# Mean and max minutes
df = pd.concat([df_w, df_l]).groupby('name')['minutes'].agg(mean_minutes='mean', max_minutes='max')
df_players = df_players.merge(df, on="name")

# Entropy
df = pd.concat([df_w, df_l])
g_sum = df.groupby('name')['minutes'].transform('sum')
values = df['minutes']/g_sum
df['minutes_entropy'] = -(values*np.log(values))
df_entropy = df.groupby('name')['minutes_entropy'].sum().reset_index()

df_players = df_players.merge(df_entropy, on="name")

C:\Users\Domenico\Anaconda3\envs\dm\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Ranking

In [13]:
df_w = df_tennis.loc[:, ['winner_name', 'winner_rank_points', 'tourney_date']].rename(columns={'winner_name': 'name', 'winner_rank_points': 'rank_points'})
df_l = (df_tennis.loc[:, ['loser_name', 'loser_rank_points', 'tourney_date']]).rename(columns={'loser_name': 'name', 'loser_rank_points': 'rank_points'})

# players and relative measures on ranking (variance, average, max)
df = pd.concat([df_l, df_w]).sort_values('tourney_date', ascending = False).groupby('name').rank_points.agg(mean_rank_points='mean', max_rank_points='max', last_rank_points='first', variance_rank_points=lambda x: np.var(x,ddof=0)).reset_index()

# add features to the player profile dataframe
df_players = df_players.merge(df, on="name")

# ratio between the last rank points and the mean rank points
df_players['lrpOnAvgrp'] = df_players['last_rank_points']/df_players['mean_rank_points']
# ratio between the last rank points and the max rank points
df_players['lrpOnMxrp'] = df_players['last_rank_points']/df_players['max_rank_points']

In [14]:
px.histogram(df_players, x="mean_rank_points").show()

In [15]:
px.histogram(df_players, x="lrpOnAvgrp").show()

### Spectator

In [16]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_spectators']].rename(columns = {'winner_name': 'name'})
df_l = (df_tennis.loc[:, ['loser_name', 'tourney_spectators']]).rename(columns = {'loser_name': 'name'})

# players and relative measures on spectators (average and max)
df = pd.concat([df_l, df_w]).groupby('name').tourney_spectators.agg(mean_tourney_spectators='mean', max_tourney_spectators='max').reset_index()

# add features to the player profile dataframe
df_players = df_players.merge(df, on="name")

### Revenue

In [17]:
df_w = df_tennis.loc[:, ['winner_name', 'tourney_revenue']].rename(columns = {'winner_name': 'name'})
df_l = (df_tennis.loc[:, ['loser_name', 'tourney_revenue']]).rename(columns = {'loser_name': 'name'})

# players and related measures on revenue (average and max)
df = pd.concat([df_l, df_w]).groupby('name').tourney_revenue.agg(mean_tourney_revenue='mean', max_tourney_revenue='max').reset_index()

# add features to the player profile dataframe
df_players = df_players.merge(df, on="name")

### Player in-match features

In [18]:
# Those feature where also used to check possible outliers, but nothing particullarry significant was found

df_tennis['rel_w_ace'] = df_tennis['w_ace']/df_tennis['w_svpt']
df_tennis['rel_l_ace'] = df_tennis['l_ace']/df_tennis['l_svpt']

df_tennis['rel_w_df'] = df_tennis['w_df']/df_tennis['w_svpt']
df_tennis['rel_l_df'] = df_tennis['l_df']/df_tennis['l_svpt']

df_tennis['rel_w_1stIn'] = df_tennis['w_1stIn']/df_tennis['w_svpt']
df_tennis['rel_l_1stIn'] = df_tennis['l_1stIn']/df_tennis['l_svpt']

df_tennis['rel_w_1stWon'] = df_tennis['w_1stWon']/df_tennis['w_svpt']
df_tennis['rel_l_1stWon'] = df_tennis['l_1stWon']/df_tennis['l_svpt']

df_tennis['rel_w_2ndWon'] = df_tennis['w_2ndWon']/df_tennis['w_svpt']
df_tennis['rel_l_2ndWon'] = df_tennis['l_2ndWon']/df_tennis['l_svpt']

df_tennis['w_1WonOn1In'] = df_tennis['w_1stWon']/df_tennis['w_1stIn']
df_tennis['l_1WonOn1In'] = df_tennis['l_1stWon']/df_tennis['l_1stIn']

df_tennis['w_1WonOnTotWon'] = df_tennis['w_1stWon']/(df_tennis['w_2ndWon'] + df_tennis['w_1stWon'])
df_tennis['l_1WonOnTotWon'] = df_tennis['l_1stWon']/(df_tennis['l_2ndWon'] + df_tennis['l_1stWon'])

df_tennis['rel_w_ptsWon'] = (df_tennis['w_1stWon'] + df_tennis['w_2ndWon'])/(df_tennis['w_1stWon'] + df_tennis['w_2ndWon'] + df_tennis['l_1stWon'] + df_tennis['l_2ndWon'])
df_tennis['rel_l_ptsWon'] = (df_tennis['l_1stWon'] + df_tennis['l_2ndWon'])/(df_tennis['w_1stWon'] + df_tennis['w_2ndWon'] + df_tennis['l_1stWon'] + df_tennis['l_2ndWon'])

df_tennis['rel_w_bpFaced'] = df_tennis['w_bpFaced']/(df_tennis['w_svpt'] + df_tennis['l_svpt'])
df_tennis['rel_l_bpFaced'] = df_tennis['l_bpFaced']/(df_tennis['w_svpt'] + df_tennis['l_svpt'])

df_tennis['rel_w_bpSaved'] = df_tennis['w_bpSaved']/df_tennis['w_bpFaced']
df_tennis['rel_l_bpSaved'] = df_tennis['l_bpSaved']/df_tennis['l_bpFaced']

df_tennis['rel_w_gmsWon'] = df_tennis['w_gmsWon']/(df_tennis['w_gmsWon'] + df_tennis['l_gmsWon'])
df_tennis['rel_l_gmsWon'] = df_tennis['l_gmsWon']/(df_tennis['w_gmsWon'] + df_tennis['l_gmsWon'])

In [19]:
df_winner = df_tennis.loc[:, ['winner_name', 'rel_w_ace', 'rel_w_df', 'rel_w_1stIn', 'rel_w_1stWon', 'rel_w_2ndWon', 'w_1WonOn1In', 'w_1WonOnTotWon', 'rel_w_ptsWon', 'rel_w_bpFaced', 'rel_w_bpSaved', 'rel_w_gmsWon']]
df_loser = df_tennis.loc[:, ['loser_name', 'rel_l_ace', 'rel_l_df', 'rel_l_1stIn', 'rel_l_1stWon', 'rel_l_2ndWon', 'l_1WonOn1In', 'l_1WonOnTotWon', 'rel_l_ptsWon', 'rel_l_bpFaced', 'rel_l_bpSaved', 'rel_l_gmsWon']]
ren_attr_list = ['name', 'rel_ace', 'rel_df', 'rel_1stIn', 'rel_1stWon', 'rel_2ndWon', '1WonOn1In', '1WonOnTotWon', 'rel_ptsWon', 'rel_bpFaced', 'rel_bpSaved', 'rel_gmsWon']

df_winner.set_axis(ren_attr_list, axis=1, inplace=True)
df_loser.set_axis(ren_attr_list, axis=1, inplace=True)
df = pd.concat([df_winner, df_loser]).groupby('name').mean().reset_index()

df_players = df_players.merge(df, on="name")

## Dropping records

#### Filter players that played at least 15 matches

In [20]:
df_winner = df_tennis.loc[:, ["winner_name"]].rename(columns={"winner_name":"name"})
df_loser = df_tennis.loc[:, ["loser_name"]].rename(columns={"loser_name":"name"})

sm15 = pd.concat([df_winner, df_loser]).groupby('name').name.count().reset_index(name="count")["count"] >= 15
sm4 = pd.concat([df_winner, df_loser]).groupby('name').name.count().reset_index(name="count")["count"] >= 4

# This will be used for classification in order to have as much data as possible
df_players_classification = df_players[sm4]
# This will be used for clustering
df_players = df_players[sm15]

df_players.iloc[:,:20].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3420 entries, 14 to 10099
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   name                       3420 non-null   object 
 1   gender                     3420 non-null   object 
 2   total_tourneys_played      3420 non-null   int64  
 3   total_matches_played       3420 non-null   int64  
 4   total_matches_won          3420 non-null   float64
 5   matches_won_ratio          3420 non-null   float64
 6   mean_performance_index     3420 non-null   Float64
 7   max_performance_index      3420 non-null   Float64
 8   min_performance_index      3420 non-null   Float64
 9   performance_index_entropy  3420 non-null   Float64
 10  ht                         454 non-null    float64
 11  age                        3289 non-null   float64
 12  hand                       3420 non-null   object 
 13  mean_minutes               1974 non-null   flo

In [21]:
# drop all records with nan values in lrpOnMxrp (that are the same for which mean_rank_points is null)
df_players.dropna(subset=['lrpOnMxrp'], inplace=True)
df_players_classification.dropna(subset=['lrpOnMxrp'], inplace=True)
df_players.iloc[:,20:].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 14 to 10099
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   lrpOnAvgrp               2997 non-null   float64
 1   lrpOnMxrp                2997 non-null   float64
 2   mean_tourney_spectators  2997 non-null   float64
 3   max_tourney_spectators   2997 non-null   float64
 4   mean_tourney_revenue     2997 non-null   float64
 5   max_tourney_revenue      2997 non-null   float64
 6   rel_ace                  1946 non-null   float64
 7   rel_df                   1946 non-null   float64
 8   rel_1stIn                1946 non-null   float64
 9   rel_1stWon               1946 non-null   float64
 10  rel_2ndWon               1946 non-null   float64
 11  1WonOn1In                1946 non-null   float64
 12  1WonOnTotWon             1946 non-null   float64
 13  rel_ptsWon               1946 non-null   float64
 14  rel_bpFaced           

## Feature selection

In [22]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 14 to 10099
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   name                       2997 non-null   object 
 1   gender                     2997 non-null   object 
 2   total_tourneys_played      2997 non-null   int64  
 3   total_matches_played       2997 non-null   int64  
 4   total_matches_won          2997 non-null   float64
 5   matches_won_ratio          2997 non-null   float64
 6   mean_performance_index     2997 non-null   Float64
 7   max_performance_index      2997 non-null   Float64
 8   min_performance_index      2997 non-null   Float64
 9   performance_index_entropy  2997 non-null   Float64
 10  ht                         453 non-null    float64
 11  age                        2973 non-null   float64
 12  hand                       2997 non-null   object 
 13  mean_minutes               1946 non-null   flo

In [23]:
df_players.head()

,name,gender,total_tourneys_played,total_matches_played,total_matches_won,matches_won_ratio,mean_performance_index,max_performance_index,min_performance_index,performance_index_entropy,...,rel_df,rel_1stIn,rel_1stWon,rel_2ndWon,1WonOn1In,1WonOnTotWon,rel_ptsWon,rel_bpFaced,rel_bpSaved,rel_gmsWon
14,abbie myers,f,81,157,75.0,0.477707,0.290166,1.0,0.0,4.59578,...,0.070857,0.578607,0.341164,0.179801,0.590180,0.654515,0.461660,0.064469,0.396978,0.506549
22,abhinav sanjeev shanmugam,m,8,16,8.0,0.500000,0.303571,0.666667,0.0,2.325714,...,0.053235,0.470209,0.341783,0.260739,0.728696,0.569641,0.508137,0.054850,0.585260,0.479098
29,abigail tere apisah,f,61,126,74.0,0.587302,0.35582,0.833333,0.0,4.545294,...,0.070367,0.581215,0.368487,0.191751,0.635412,0.663208,0.544618,0.064519,0.576609,0.566284
36,adam el mihdawy,m,17,23,6.0,0.260870,0.086957,0.166667,0.0,2.484907,...,0.119067,0.450848,0.326516,0.221596,0.719894,0.592229,0.472459,0.062262,0.486949,0.415704
43,adam pavlasek,m,95,201,109.0,0.542289,0.315535,0.833333,0.0,4.919851,...,0.033647,0.608423,0.418225,0.196069,0.686969,0.679038,0.505872,0.047200,0.578063,0.514447


### Filling age missing values

In [24]:
# filling the 4 missing value for attribute age with the mean
means = df_players['age'].mean()
df_players['age'] = df_players['age'].fillna(means)
df_players.iloc[:,:20].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 14 to 10099
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   name                       2997 non-null   object 
 1   gender                     2997 non-null   object 
 2   total_tourneys_played      2997 non-null   int64  
 3   total_matches_played       2997 non-null   int64  
 4   total_matches_won          2997 non-null   float64
 5   matches_won_ratio          2997 non-null   float64
 6   mean_performance_index     2997 non-null   Float64
 7   max_performance_index      2997 non-null   Float64
 8   min_performance_index      2997 non-null   Float64
 9   performance_index_entropy  2997 non-null   Float64
 10  ht                         453 non-null    float64
 11  age                        2997 non-null   float64
 12  hand                       2997 non-null   object 
 13  mean_minutes               1946 non-null   flo

### Correlation analysis

Here we look at the correlation between all the defined features

In [25]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

corr_threshold = 0.8
correlation = df_players.corr(method="pearson") #.abs()
correlation = correlation.where(np.tril(np.ones(correlation.shape)).astype(bool)) # remove upper triangle

px.imshow(correlation, labels=dict(color="Correlation"), color_continuous_scale=px.colors.diverging.RdBu, zmin=-1, zmax=1, width=1000, height=1000).show()

correlation = correlation.unstack().drop(labels=get_redundant_pairs(correlation)).sort_values(ascending=False).drop_duplicates()
print(f"These are {len(correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)])} pairs whose correlation is bigger/small than ±{corr_threshold*100}%:")
correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)]

These are 19 pairs whose correlation is bigger/small than ±80.0%:


mean_tourney_spectators  mean_tourney_revenue         0.987819
total_matches_played     total_matches_won            0.981810
total_tourneys_played    total_matches_played         0.976075
max_tourney_spectators   max_tourney_revenue          0.961528
mean_rank_points         max_rank_points              0.961302
matches_won_ratio        mean_performance_index       0.954152
                         rel_gmsWon                   0.925484
total_tourneys_played    total_matches_won            0.918178
total_matches_played     performance_index_entropy    0.905295
total_matches_won        performance_index_entropy    0.896947
max_rank_points          last_rank_points             0.876132
total_tourneys_played    performance_index_entropy    0.872427
mean_performance_index   rel_gmsWon                   0.871715
minutes_entropy          max_tourney_spectators       0.830638
mean_performance_index   max_performance_index        0.819540
rel_1stWon               1WonOn1In                    0

### Selection

The following feature may provide an interesting picture about the performance of the players

In [26]:
feautures = ['lrpOnAvgrp', 'lrpOnMxrp', 'mean_performance_index', 'matches_won_ratio', 'mean_rank_points', 'max_rank_points', 'last_rank_points', 'variance_rank_points', 'mean_tourney_spectators', 'max_tourney_spectators', 'mean_tourney_revenue', 'max_tourney_revenue', 'rel_ptsWon', 'rel_gmsWon']
df_selected = df_players[feautures].reset_index(drop=True)

corr_threshold = 0.7
correlation = df_selected.corr(method="pearson") #.abs()
correlation = correlation.where(np.tril(np.ones(correlation.shape)).astype(bool)) # remove upper triangle

px.imshow(correlation, labels=dict(color="Correlation"), color_continuous_scale=px.colors.diverging.RdBu, zmin=-1, zmax=1, width=1000, height=1000).show()

correlation = correlation.unstack().drop(labels=get_redundant_pairs(correlation)).sort_values(ascending=False).drop_duplicates()
print(f"These are {len(correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)])} pairs whose correlation is bigger/small than ±{corr_threshold*100}%:")
correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)]

These are 15 pairs whose correlation is bigger/small than ±70.0%:


mean_tourney_spectators  mean_tourney_revenue      0.987819
max_tourney_spectators   max_tourney_revenue       0.961528
mean_rank_points         max_rank_points           0.961302
mean_performance_index   matches_won_ratio         0.954152
matches_won_ratio        rel_gmsWon                0.925484
max_rank_points          last_rank_points          0.876132
mean_performance_index   rel_gmsWon                0.871715
mean_rank_points         last_rank_points          0.796869
mean_tourney_revenue     max_tourney_revenue       0.795775
mean_tourney_spectators  max_tourney_spectators    0.795617
max_tourney_spectators   mean_tourney_revenue      0.793692
mean_tourney_spectators  max_tourney_revenue       0.783502
lrpOnAvgrp               lrpOnMxrp                 0.775849
max_rank_points          variance_rank_points      0.773908
mean_rank_points         variance_rank_points      0.733196
dtype: float64

The idea is to drop all the features that have more than `70%` of correlation

In [27]:
feautures = ['lrpOnMxrp', 'matches_won_ratio', 'mean_rank_points', 'mean_tourney_spectators', 'max_tourney_revenue', 'rel_ptsWon']
df_selected = df_players[feautures].reset_index(drop=True)

corr_threshold = 0.7
correlation = df_selected.corr(method="pearson") #.abs()
correlation = correlation.where(np.tril(np.ones(correlation.shape)).astype(bool)) # remove upper triangle

px.imshow(correlation, labels=dict(color="Correlation"), color_continuous_scale=px.colors.diverging.RdBu, zmin=-1, zmax=1, width=1000, height=1000).show()

correlation = correlation.unstack().drop(labels=get_redundant_pairs(correlation)).sort_values(ascending=False).drop_duplicates()
print(f"These are {len(correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)])} pairs whose correlation is bigger/small than ±{corr_threshold*100}%:")
correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)]

These are 1 pairs whose correlation is bigger/small than ±70.0%:


mean_tourney_spectators  max_tourney_revenue    0.783502
dtype: float64

In [28]:
df_players[feautures].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2997 entries, 14 to 10099
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   lrpOnMxrp                2997 non-null   float64
 1   matches_won_ratio        2997 non-null   float64
 2   mean_rank_points         2997 non-null   float64
 3   mean_tourney_spectators  2997 non-null   float64
 4   max_tourney_revenue      2997 non-null   float64
 5   rel_ptsWon               1946 non-null   float64
dtypes: float64(6)
memory usage: 163.9 KB


The idea is to drop `rel_ptsWon` becuase there are only 1946 non-null values. And hence these are the final selected features

In [29]:
feautures = ['lrpOnMxrp', 'matches_won_ratio', 'mean_rank_points', 'mean_tourney_spectators', 'max_tourney_revenue']
df_selected = df_players[feautures].reset_index(drop=True)
df_selected.describe()

,lrpOnMxrp,matches_won_ratio,mean_rank_points,mean_tourney_spectators,max_tourney_revenue
count,2997.000000,2997.000000,2997.000000,2997.000000,2.997000e+03
mean,0.689888,0.476883,203.446610,3804.478131,2.178712e+06
std,0.315442,0.121423,529.121629,995.986197,1.575108e+06
min,0.005181,0.000000,1.000000,1247.840000,6.206034e+05
25%,0.417984,0.407407,12.833333,3171.878788,9.922213e+05
50%,0.795918,0.491525,48.956522,3331.500000,1.244892e+06
75%,1.000000,0.560166,174.029126,4264.965909,3.749463e+06
max,1.000000,0.852507,10235.315634,10932.342105,5.002794e+06


In [30]:
df_selected = df_players[feautures].reset_index(drop=True)

corr_threshold = 0.7
correlation = df_selected.corr(method="pearson") #.abs()
correlation = correlation.where(np.tril(np.ones(correlation.shape)).astype(bool)) # remove upper triangle

px.imshow(correlation, labels=dict(color="Correlation"), color_continuous_scale=px.colors.diverging.RdBu, zmin=-1, zmax=1, width=1000, height=1000).show()

correlation = correlation.unstack().drop(labels=get_redundant_pairs(correlation)).sort_values(ascending=False).drop_duplicates()
print(f"These are {len(correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)])} pairs whose correlation is bigger/small than ±{corr_threshold*100}%:")
correlation[(correlation>=corr_threshold) | (correlation<=-corr_threshold)]

These are 1 pairs whose correlation is bigger/small than ±70.0%:


mean_tourney_spectators  max_tourney_revenue    0.783502
dtype: float64

### Analysis

By looking at the graphs, it's fair to say that during the clustering phase the features need to be standardized and outliers could be managed

In [31]:
for feature in feautures:
  px.histogram(df_players, x=feature, marginal="box").show()

## Output

In [32]:
df_players.to_csv("./datasets/players.csv")
df_players_classification.to_csv("./datasets/players_classification.csv")